In [1]:
!pip install scrapy

In [2]:
import scrapy
import re
import csv

class News_Spider(scrapy.Spider):
    name="news_spider"

    def start_requests(self):
        url="https://www.irishtimes.com/politics/"
        yield scrapy.Request(url=url,callback=self.parse_front)

    def parse_front(self,response):
        #Code to parse Irish Times Politics Page
        date_reg_exp = re.compile('[-/]politics[-/]\d{4}[-/]\d{2}[-/]\d{2}')
        article_links = response.css('body>div a::attr(href)').extract()
        article_links_cleaned=[]
        
        for link in article_links:
            matches_list=date_reg_exp.findall(link)
            if len(matches_list)>0:
                link_to_add="https://www.irishtimes.com"+link
                if link_to_add in article_links:
                    continue
                article_links_cleaned.append(link_to_add)
                
        for link_to_follow in article_links_cleaned:
            yield response.follow(url=link_to_follow,callback=self.get_title_politics_section)

    def get_title_politics_section(self,response):
        # Extract title from the response using CSS.
        title = response.css('h1::text').get()
        url = response.url

        # Write title and URL to a CSV file.
        with open('titles.csv', mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([title, url])

In [3]:
from scrapy.crawler import CrawlerProcess
process=CrawlerProcess()
process.crawl(News_Spider)
process.start()

2023-02-25 11:38:04 [scrapy.utils.log] INFO: Scrapy 2.6.2 started (bot: scrapybot)
2023-02-25 11:38:04 [scrapy.utils.log] INFO: Versions: lxml 4.9.1.0, libxml2 2.9.14, cssselect 1.1.0, parsel 1.6.0, w3lib 1.21.0, Twisted 22.2.0, Python 3.9.16 (main, Jan 11 2023, 10:02:19) - [Clang 14.0.6 ], pyOpenSSL 22.0.0 (OpenSSL 1.1.1t  7 Feb 2023), cryptography 38.0.4, Platform macOS-10.16-x86_64-i386-64bit
2023-02-25 11:38:04 [scrapy.crawler] INFO: Overridden settings:
{}
2023-02-25 11:38:04 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2023-02-25 11:38:04 [scrapy.extensions.telnet] INFO: Telnet Password: 0ded1baa0dd09008
2023-02-25 11:38:04 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2023-02-25 11:38:04 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddleware